In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# 합성곱 신경망

### 데이터셋 정보
- MNIST 데이터셋
- 훈련 데이터와 테스트 데이터가 각각 25000개
- 데이터의 각 review에는 label이 붙어 있다.
- 부정은 0, 긍정은 1로 나타낸다.
- 약 50%는 긍정, 50%는 부정 리뷰로 구성
- review 문장의 단어들을 출현빈도순으로 정렬해서 정수로 변환시킨 시퀀스를 x로 한다.
- 스탠포드 대학의 앤드류 마스가 수집했다.

### 예측하고자 하는 방법
- 이진 분류 문제로, 학습 데이터를 이용해, 설계한 신경망을 학습시킨다.
- 크로스엔트로피를 이용해 원본 분포와 예측 분포 사이를 측정한다.
- rmsprop 옵티마이저와 categorical_crossentropy 손실 함수로 모델을 설정
- 학습된 신경망을 이용해 어떤 리뷰가 긍정일 확률을 예측할 수 있도록 한다.

### 사용된 모델

- Conv2D: 한 원소의 앞뒤 원소들을 확인(지역적인 범위를 파악)해 큰 feature를 뽑아내는 layer
- MaxPooling2D: 지역적 범위에서 사소한 변화로 인한 영향을 줄여주는 layer

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

W0812 14:21:10.855482 17056 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 14:21:10.923321 17056 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 14:21:10.944265 17056 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 14:21:10.983161 17056 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

W0812 14:21:23.759767 17056 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0812 14:21:23.783235 17056 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0812 14:21:23.888940 17056 deprecation.py:323] From C:\Users\a9327\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0812 14:21:23.975707 17056 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v

Epoch 1/5
60000/60000 [==============================] - 27s 455us/step - loss: 0.1735 - acc: 0.9447
Epoch 2/5
60000/60000 [==============================] - 29s 485us/step - loss: 0.0474 - acc: 0.9853
Epoch 3/5
60000/60000 [==============================] - 29s 491us/step - loss: 0.0335 - acc: 0.9895
Epoch 4/5
60000/60000 [==============================] - 29s 484us/step - loss: 0.0258 - acc: 0.9917
Epoch 5/5
60000/60000 [==============================] - 29s 484us/step - loss: 0.0199 - acc: 0.9938


In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 109us/step


In [10]:
test_acc

0.9916